In [1]:
import pandas as pd
import sklearn as sklearn
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

from utils.randomized_search import logUnifD
from utils.base_set import X_train, y_train, seed

### KNN 

In [30]:
model = KNeighborsClassifier()
K = X_train.shape[0]
k = 5
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
param = {
    'n_neighbors': logUnifD(10, K),
    'weights': ['uniform', 'distance'],
    #'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': stats.randint(1, 4), # minkowski metric
}
n = 100
classifier_KNN = RandomizedSearchCV(estimator=model, 
                                param_distributions=param,
                                cv=cv,
                                n_iter=n,
                                scoring="roc_auc", 
                                random_state=seed)
classifier_KNN.fit(X_train,y_train)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 359, n_neighbors = 434

In [29]:
columns_to_keep = ['param_n_neighbors', 
                   'param_weights', 
                   #'param_algorithm',
                   'param_p', 
                   'mean_test_score',
                   'rank_test_score']
pd.DataFrame(classifier_KNN.cv_results_).sort_values('rank_test_score')[columns_to_keep]

,param_n_neighbors,param_weights,param_p,mean_test_score,rank_test_score
6,15,distance,1,0.839120,1
19,11,distance,1,0.838640,2
0,14,distance,1,0.837595,3
64,13,distance,1,0.836097,4
70,33,distance,1,0.831356,5
...,...,...,...,...,...
12,173,distance,3,0.751793,96
93,148,uniform,3,0.745932,97
80,153,uniform,3,0.745410,98
32,176,uniform,3,0.744043,99


In [4]:
best = classifier_KNN.best_params_
auc_roc = classifier_KNN.best_score_
print(best)
print(auc_roc)

{'n_neighbors': 15, 'p': 1, 'weights': 'distance'}
0.8391199042391625


In [31]:
X_train.shape

(449, 200)